In [1]:
import torch
from torch import nn

In [2]:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

print(device)

cuda


In [3]:
from sklearn import datasets

In [4]:
wine = datasets.load_wine()
data = wine.data
target = wine.target

In [5]:
print(data.shape)

(178, 13)


In [6]:
print(target.shape)

(178,)


In [7]:
print(wine.feature_names)

['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline']


In [8]:
wine.target_names

array(['class_0', 'class_1', 'class_2'], dtype='<U7')

In [9]:
class WineClassifier(nn.Module):

  def __init__(self, input_size, hidden_size, out_size):
    super(WineClassifier, self).__init__()

    self.hidden = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.out = nn.Linear(hidden_size, out_size)
    self.softmax = nn.Softmax()

  def forward(self, X):

    feature = self.relu(self.hidden(X))
    output = self.softmax(self.out(feature))

    return output

input_size = data.shape[1]
hidden_size = 32
out_size = len(wine.target_names)
net = WineClassifier(input_size, hidden_size, out_size).to(device)

In [10]:
print(net)

WineClassifier(
  (hidden): Linear(in_features=13, out_features=32, bias=True)
  (relu): ReLU()
  (out): Linear(in_features=32, out_features=3, bias=True)
  (softmax): Softmax(dim=None)
)


In [11]:
criterion = nn.CrossEntropyLoss().to(device)

In [12]:
criterion

CrossEntropyLoss()

In [13]:
Xtns = torch.from_numpy(data).float()
Ytns = torch.from_numpy(target)

Xtns = Xtns.to(device)
Ytns = Ytns.to(device)

print(Xtns.dtype)
print(Ytns.dtype)

torch.float32
torch.int64


In [14]:
pred = net(Xtns)
print(pred.shape)

torch.Size([178, 3])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


In [15]:
print(pred)

tensor([[0.0000e+00, 4.3489e-10, 1.0000e+00],
        [0.0000e+00, 8.5280e-09, 1.0000e+00],
        [0.0000e+00, 4.2390e-09, 1.0000e+00],
        [0.0000e+00, 1.5234e-10, 1.0000e+00],
        [2.9394e-34, 1.9588e-08, 1.0000e+00],
        [0.0000e+00, 1.8795e-10, 1.0000e+00],
        [0.0000e+00, 2.9133e-09, 1.0000e+00],
        [0.0000e+00, 2.4897e-10, 1.0000e+00],
        [0.0000e+00, 1.3219e-08, 1.0000e+00],
        [0.0000e+00, 1.6420e-08, 1.0000e+00],
        [0.0000e+00, 2.7789e-10, 1.0000e+00],
        [0.0000e+00, 4.6400e-09, 1.0000e+00],
        [0.0000e+00, 6.3874e-09, 1.0000e+00],
        [0.0000e+00, 8.5791e-09, 1.0000e+00],
        [0.0000e+00, 1.9101e-10, 1.0000e+00],
        [0.0000e+00, 6.8650e-10, 1.0000e+00],
        [0.0000e+00, 3.5200e-10, 1.0000e+00],
        [0.0000e+00, 1.9035e-09, 1.0000e+00],
        [0.0000e+00, 7.4098e-11, 1.0000e+00],
        [1.2351e-39, 7.2681e-09, 1.0000e+00],
        [2.0516e-36, 3.7389e-09, 1.0000e+00],
        [3.6320e-36, 7.1569e-08, 1

In [16]:
loss = criterion(pred, Ytns)

In [17]:
print(loss)

tensor(1.2818, device='cuda:0', grad_fn=<NllLossBackward0>)


# Regressão

In [18]:
diabetes = datasets.load_diabetes()
data = diabetes.data
target = diabetes.target

In [19]:
class WineClassifier(nn.Module):

  def __init__(self, input_size, hidden_size, out_size):
    super(WineClassifier, self).__init__()

    self.hidden = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.out = nn.Linear(hidden_size, out_size)
    self.softmax = nn.Softmax(dim=-1)

  def forward(self, X):

    feature = self.relu(self.hidden(X))
    output = self.softmax(self.out(feature))

    return output

input_size = data.shape[1]
hidden_size = 32
out_size = 1

net = WineClassifier(input_size, hidden_size, out_size).to(device)

In [20]:
criterion = nn.MSELoss().to(device)

Xtns = torch.from_numpy(data).float().to(device)
Ytns = torch.from_numpy(target).float().to(device)

In [21]:
print(Xtns.shape)

torch.Size([442, 10])


In [22]:
print(Ytns.shape)

torch.Size([442])


In [23]:
pred = net(Xtns)

In [24]:
pred.shape

torch.Size([442, 1])

In [25]:
loss = criterion(pred.squeeze(), Ytns)

In [26]:
print(loss.data)

tensor(28771.2168, device='cuda:0')


# otimização

In [27]:
import matplotlib.pyplot as plt

In [28]:
wine = datasets.load_wine()
features = [0,9]
data = wine.data[:, features]
targets = wine.target

In [29]:
from sklearn.preprocessing import StandardScaler

In [30]:
scaler = StandardScaler()

In [31]:
data = scaler.fit_transform(data)

In [32]:
input_size = data.shape[1]
hidden_size = 32
out_size = len(wine.target_names)

In [33]:
nn = nn.Sequential(
    nn.Linear(input_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, out_size),
    nn.Softmax()
)

net = net.to(device)

In [34]:
net

WineClassifier(
  (hidden): Linear(in_features=10, out_features=32, bias=True)
  (relu): ReLU()
  (out): Linear(in_features=32, out_features=1, bias=True)
  (softmax): Softmax(dim=-1)
)

In [35]:
device

device(type='cuda')

In [36]:
from torch import optim

# função de perda
# criterion = nn.CrossEntropyLoss().to(device)

# otimizador
optmizer = optim.SGD(net.parameters(), lr=1e-3)

In [37]:
X = torch.FloatTensor(data).to(device)
Y = torch.LongTensor(targets).to(device)

In [ ]:
pred = net(X)
loss = criterion(pred,Y)

loss.backward()
optmizer.step()